<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Project2024DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download paultimothymooney/chest-xray-pneumonia
! unzip /content/chest-xray-pneumonia.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open /content/chest-xray-pneumonia.zip, /content/chest-xray-pneumonia.zip.zip or /content/chest-xray-pneumonia.zip.ZIP.


# Project Notebook :)

## Main ideas

* time: 10.jan.2024 - 5.feb.2024
* has to be simple
* maybe somethings about biology

### Progress

I have downloaded a [chest X-ray ds](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data) that can be used to identify pneumonia patients. I want to create a NN that classifies pediatric pnuemonia.

**Current objectives:**
* Wacth 2 lectures on NN testing and how to work with images

**Feature objectives:**

1. Preprocess data
1. If needed seprate into train-test-val
1. Choose an architecure
1. Train model
1. Evaluate model based on validation sets
  * If possible, find another ds and validate results!
1. Test model to choose the best one
